In [1]:
%pip install langgraph
%pip install langchain-community
%pip install langchain-ollama
%pip install duckduckgo-search
%pip install ollama

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
# from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults
# from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
from abc import ABC,abstractmethod
import ollama

In [3]:
_ = load_dotenv()

In [4]:
tool = TavilySearchResults()
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [ ]:
# wrapper = DuckDuckGoSearchAPIWrapper(max_results=5)
# tool = DuckDuckGoSearchResults(api_wrapper=wrapper)
# print(type(tool))
# print(tool.name)

In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [6]:
class Agent:

    def __init__(self,model,tools,system_message=""):
        self.system = system_message
        self.graph = self._initialize_graph()
        self.tools = {t.name:t for t in tools}
        self.model = model.bind_tools(tools)
    
    def _initialize_graph(self):
        graph = StateGraph(AgentState)
        graph.add_node('llm',self.call_llm)
        graph.add_node('action',self.take_action)
        graph.add_conditional_edges('llm',self.exists_action,{True:'action',False:END})
        graph.add_edge('action','llm')
        graph.set_entry_point('llm')
        return graph.compile()

    def call_llm(self,state:AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages':[message]}

    def take_action(self,state:AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for tool in tool_calls:
            print(f"Calling :{tool}")
            if not tool['name'] in self.tools:
                print(f"Tool {tool['name']} not found")
                result = "Bad tool name, retry"
            else:
                result = self.tools[tool['name']].invoke(tool['args'])
            results.append(ToolMessage(tool_call_id=tool['id'],name=tool['name'],content=str(result)))
        print("Back to the model.")
        return {'messages':results}

    def exists_action(self,state:AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

In [7]:
try:
    ollama.show('llama3.2')
except ollama.ResponseError as e:
    ollama.pull('llama3.2')
    ollama.show('llama3.2')

In [8]:
system_prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [9]:
llm = ChatOllama(model="llama3.2")

In [10]:
abot = Agent(model=llm,tools=[tool],system_message=system_prompt)

In [11]:
messages = [HumanMessage(content="What is the temperature of kodaikanal?")]
result = abot.graph.invoke({"messages": messages})
result

Calling :{'name': 'tavily_search_results_json', 'args': {'query': 'Kodaikanal temperature'}, 'id': '9682612f-a98d-4dc4-98f2-313015f192f2', 'type': 'tool_call'}
Back to the model.


{'messages': [HumanMessage(content='What is the temperature of kodaikanal?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-10-03T05:46:44.6863148Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'Kodaikanal temperature'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 6787284800, 'load_duration': 4515730000, 'prompt_eval_count': 261, 'prompt_eval_duration': 1325284000, 'eval_count': 25, 'eval_duration': 939538000}, id='run-7d589e10-cdfd-43df-93ad-9ecaa264a3d9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Kodaikanal temperature'}, 'id': '9682612f-a98d-4dc4-98f2-313015f192f2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 261, 'output_tokens': 25, 'total_tokens': 286}),
  ToolMessage(content="[{'url': 'https://weather.com/en-IN/weather/tenday/l/K